In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Define the CNN model class
class AllConvNet(nn.Module):
    def __init__(self):
        super(AllConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding='same')
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding='same')
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding='same')
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding='same')
        self.conv5 = nn.Conv2d(512, 10, kernel_size=3, padding='same')
        self.relu = nn.LeakyReLU()
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.avg_pool(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.avg_pool(x)
        x = self.relu(self.conv5(x))
        x = self.relu(self.global_avg_pool(x)).squeeze()
        return x

# Create an instance of the model and move it to the GPU(s)
device = torch.device("cuda:0")  # Use GPU 0
net = AllConvNet().to(device)
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Training loop
num_epochs = 10
loss_history = []
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    loss_history.append(running_loss / (i + 1))
    print(f'Epoch {epoch+1}, Loss: {running_loss / (i+1)}')

print('Finished Training')

# Test the model
correct = 0
total = 0
net.eval()  # Set the model to evaluation mode
all_labels = []
all_preds = []

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().tolist())
        all_preds.extend(predicted.cpu().tolist())

print(f'Accuracy on the test dataset: {100 * correct / total:.2f}%')



# Select a GPU for image display
torch.cuda.device(0)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.8221961775856554
Epoch 2, Loss: 1.109287455258772
Epoch 3, Loss: 0.8386700872493826
Epoch 4, Loss: 0.6859789059290191
Epoch 5, Loss: 0.5789740541783135
Epoch 6, Loss: 0.49474102843676687
Epoch 7, Loss: 0.4203293763310708
Epoch 8, Loss: 0.35642744069132964
Epoch 9, Loss: 0.29996956993475593
Epoch 10, Loss: 0.24670053943229453
Finished Training
Accuracy on the test dataset: 80.07%


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check for GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the Transformer model for image classification
class VisionTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, patch_size, num_classes, dropout=0.1):
        super(VisionTransformer, self).__init__()

        self.patch_embedding = nn.Conv2d(input_dim, hidden_dim, kernel_size=patch_size, stride=patch_size)

        num_patches = (32 // patch_size) * (32 // patch_size)
        self.positional_embedding = nn.Parameter(torch.randn(1, num_patches + 1, hidden_dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        self.fc = nn.Linear(hidden_dim, num_classes)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.patch_embedding(x).flatten(2).transpose(1, 2)
        batch_size = x.size(0)
        x = torch.cat([self.cls_token.expand(batch_size, -1, -1), x], dim=1)
        x += self.positional_embedding
        x = self.dropout(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.fc(x)
        return x

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Create an instance of the Vision Transformer model
transformer_model = VisionTransformer(input_dim=3, hidden_dim=256, num_heads=8, num_layers=6, patch_size=8, num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    transformer_model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = transformer_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(trainloader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0  # Reset running loss

    print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

# Evaluate the model
transformer_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = transformer_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test dataset: {100 * correct / total:.2f}%')



Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/10], Batch [100/782], Loss: 2.3831
Epoch [1/10], Batch [200/782], Loss: 2.3333
Epoch [1/10], Batch [300/782], Loss: 2.3241
Epoch [1/10], Batch [400/782], Loss: 2.3216
Epoch [1/10], Batch [500/782], Loss: 2.3189
Epoch [1/10], Batch [600/782], Loss: 2.3204
Epoch [1/10], Batch [700/782], Loss: 2.3162
Epoch 1, Loss: 0.24275256270337897
Epoch [2/10], Batch [100/782], Loss: 2.3137
Epoch [2/10], Batch [200/782], Loss: 2.3160
Epoch [2/10], Batch [300/782], Loss: 2.3159
Epoch [2/10], Batch [400/782], Loss: 2.3133
Epoch [2/10], Batch [500/782], Loss: 2.3098
Epoch [2/10], Batch [600/782], Loss: 2.3108
Epoch [2/10], Batch [700/782], Loss: 2.3111
Epoch 2, Loss: 0.24213888761027694
Epoch [3/10], Batch [100/782], Loss: 2.3101
Epoch [3/10], Batch [200/782], Loss: 2.3100
Epoch [3/10], Batch [300/782], Loss: 2.3083
Epoch [3/10], Batch [400/782], Loss: 2.3106
Epoch [3/10], Batch [500/782], Loss: 2.3083
Epoch [3/10], Batch [600/782], Loss: 2.3078
Epoch [3/10], Batch [700/782], Loss: 2.3108
Epoch 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check for GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the Transformer encoder model class
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim, dropout)

        encoder_layers = nn.TransformerEncoderLayer(hidden_dim, num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc = nn.Linear(hidden_dim, 10)  # Output for CIFAR-10 classes

    def forward(self, x):
        x = x.view(x.size(0), -1, 3)  # Reshape to (batch_size, height * width, channels)
        x = self.embedding(x)
        x = self.positional_encoding(x.permute(1, 0, 2))  # Reshape and apply positional encoding
        encoder_output = self.transformer_encoder(x)
        output = self.fc(encoder_output[-1])  # Take the output from the last layer
        return output

# Define the positional encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Create an instance of the Transformer encoder model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformer_model = TransformerEncoder(input_dim=3, hidden_dim=256, num_heads=8, num_layers=6).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer_model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    transformer_model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.view(-1, 3, 32 * 32).to(device), labels.to(device)  # Flatten images
        optimizer.zero_grad()
        outputs = transformer_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(trainloader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0  # Reset running loss

    print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

# Evaluate the model
transformer_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.view(-1, 3, 32 * 32).to(device), labels.to(device)  # Flatten images
        outputs = transformer_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test dataset: {100 * correct / total:.2f}%')



Files already downloaded and verified
Files already downloaded and verified
Epoch [1/2], Batch [100/782], Loss: 2.3837
Epoch [1/2], Batch [200/782], Loss: 2.3352
Epoch [1/2], Batch [300/782], Loss: 2.3253
Epoch [1/2], Batch [400/782], Loss: 2.3225
Epoch [1/2], Batch [500/782], Loss: 2.3192
Epoch [1/2], Batch [600/782], Loss: 2.3224
Epoch [1/2], Batch [700/782], Loss: 2.3132
Epoch 1, Loss: 0.24270789153740535
Epoch [2/2], Batch [100/782], Loss: 2.3116
Epoch [2/2], Batch [200/782], Loss: 2.3073
Epoch [2/2], Batch [300/782], Loss: 2.3115
Epoch [2/2], Batch [400/782], Loss: 2.3092
Epoch [2/2], Batch [500/782], Loss: 2.3092
Epoch [2/2], Batch [600/782], Loss: 2.3085
Epoch [2/2], Batch [700/782], Loss: 2.3079
Epoch 2, Loss: 0.24174555061418382
Finished Training
Accuracy on the test dataset: 10.00%
